In [1]:
# importing packages an dependencies
#%tensorflow_version 1.x

import cv2
import json
import os

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path
from tqdm import tqdm

#from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization, Activation
from keras.layers import Convolution2D, MaxPooling2D

Using TensorFlow backend.


In [37]:
import os
import zipfile
#import imageio as io

from keras.preprocessing import image
from keras import backend as K
from keras.applications.vgg16 import preprocess_input

In [38]:
import imageio

In [39]:
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout

In [40]:
from keras.callbacks import *

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
            base = 0.001 max = 0.006
    """

    def __init__(self, base_lr=0.0001, max_lr=0.01, step_size=300., mode='exp_range',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [41]:
from keras.callbacks import *

clr_triangular = CyclicLR(mode='exp_range')

In [42]:
os.chdir("F:\\EIP\\hvc_data\\")

In [43]:
df = pd.read_csv("./hvc_annotations.csv")
print(df.shape)

(13573, 10)


In [44]:
#display sample images
#from google.colab.patches import cv2_imshow
ext_img = imageio.imread("resized/8335.jpg")
print(ext_img.shape)

(224, 224, 3)


In [45]:
one_hot_df = pd.concat([
                        df[['image_path']],
                        pd.get_dummies(df['gender'], prefix = "gender"),
                        pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose")
                        ], axis = 1)
one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [46]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([imageio.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [47]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15,random_state = 111)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [48]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,...,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
2249,resized/2250.jpg,1,0,0,1,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0,1
5664,resized/5665.jpg,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
5608,resized/5609.jpg,1,0,1,0,0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,0
4157,resized/4158.jpg,0,1,0,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,0
7254,resized/7255.jpg,0,1,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,1,0,0


In [49]:
train_gen = PersonDataGenerator(train_df, batch_size=32)
valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False)

In [50]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = {k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'gender': 2,
 'image_quality': 3,
 'age': 5,
 'weight': 4,
 'bag': 3,
 'pose': 3,
 'footwear': 3,
 'emotion': 4}

In [51]:
losses = {"gender_output" : "binary_crossentropy",
         "image_quality_output" : "categorical_crossentropy",
         "age_output" : "categorical_crossentropy",
         "weight_output" : "categorical_crossentropy",
         "bag_output" : "categorical_crossentropy",
         "footwear_output" : "categorical_crossentropy",
         "pose_output" : "categorical_crossentropy",
         "emotion_output" : "categorical_crossentropy"}
lossWeights = {"gender_output" : 1.0,
         "image_quality_output" : 1.0,
         "age_output" : 1.0,
         "weight_output" : 1.0,
         "bag_output" : 1.0,
         "footwear_output" : 1.0,
         "pose_output" : 1.0,
         "emotion_output" : 1.0}

In [52]:
#######inceptionV3### MOD

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

backend = keras.backend
layers = keras.layers
models = keras.models
keras_utils = keras.utils

In [53]:
from keras import applications
from keras_applications.imagenet_utils import _obtain_input_shape

In [54]:
def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):
    """Utility function to apply conv + BN.
    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.
    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if backend.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = layers.Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = layers.BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = layers.Activation('relu', name=name)(x)
    return x

In [55]:
def InceptionV3(include_top=True,
                input_tensor=None,
                input_shape=None,
                **kwargs):
    """Instantiates the Inception v3 architecture.
    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(299, 299, 3)` (with `channels_last` data format)
            or `(3, 299, 299)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 75.
            E.g. `(150, 150, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional block.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional block, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    global backend, layers, models, keras_utils
    weights = None 
    

    if not (weights in {'imagenet', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    backend = keras.backend
    layers = keras.layers
    models = keras.models
    keras_utils = keras.utils
    
    input_shape = _obtain_input_shape(
        input_shape,
        default_size=299,
        min_size=75,
        data_format=backend.image_data_format(),
        require_flatten=include_top,
        weights=weights)

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    if backend.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3

    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 80, 1, 1, padding='valid')
    x = conv2d_bn(x, 192, 3, 3, padding='valid')
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = layers.AveragePooling2D((3, 3),
                                          strides=(1, 1),
                                          padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')

    # mixed 1: 35 x 35 x 288
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = layers.AveragePooling2D((3, 3),
                                          strides=(1, 1),
                                          padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')

    # mixed 2: 35 x 35 x 288
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = layers.AveragePooling2D((3, 3),
                                          strides=(1, 1),
                                          padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = layers.concatenate(
        [branch3x3, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed3')

    # mixed 4: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = layers.AveragePooling2D((3, 3),
                                          strides=(1, 1),
                                          padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=channel_axis,
        name='mixed4')

    # mixed 5, 6: 17 x 17 x 768
    for i in range(2):
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 160, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 160, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = layers.AveragePooling2D(
            (3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = layers.concatenate(
            [branch1x1, branch7x7, branch7x7dbl, branch_pool],
            axis=channel_axis,
            name='mixed' + str(5 + i))

    # mixed 7: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 192, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 192, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = layers.AveragePooling2D((3, 3),
                                          strides=(1, 1),
                                          padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=channel_axis,
        name='mixed7')

    # mixed 8: 8 x 8 x 1280
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,
                          strides=(2, 2), padding='valid')

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn(
        branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = layers.concatenate(
        [branch3x3, branch7x7x3, branch_pool],
        axis=channel_axis,
        name='mixed8')

    # mixed 9: 8 x 8 x 2048
    for i in range(2):
        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = layers.concatenate(
            [branch3x3_1, branch3x3_2],
            axis=channel_axis,
            name='mixed9_' + str(i))

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = layers.concatenate(
            [branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

        branch_pool = layers.AveragePooling2D(
            (3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = layers.concatenate(
            [branch1x1, branch3x3, branch3x3dbl, branch_pool],
            axis=channel_axis,
            name='mixed' + str(9 + i))

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = keras_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = models.Model(inputs, x, name='inception_v3')
    return model

In [56]:
basemodel = InceptionV3(input_shape = (224,224,3))

In [60]:
def build_tower(layer,numclasses):
    x = Dropout(0.2)(layer)
    x = Convolution2D(filters=numclasses,kernel_size = (1,1),activation='relu',use_bias=False,kernel_initializer='he_uniform')(x)
    x = GlobalAveragePooling2D()(x)
    return x
  #x = Activation('softmax')  

In [61]:
neck = basemodel.output

def build_head(name, in_layer):
    return Activation(activation="softmax", name=f"{name}_output")(in_layer)

gender = build_head("gender", build_tower(neck,2))
image_quality = build_head("image_quality", build_tower(neck,3))
age = build_head("age", build_tower(neck,5))
weight = build_head("weight", build_tower(neck,4))
bag = build_head("bag", build_tower(neck,3))
footwear = build_head("footwear", build_tower(neck,3))
emotion = build_head("emotion", build_tower(neck,4))
pose = build_head("pose", build_tower(neck,3))


model2 = Model(
    inputs=basemodel.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [62]:
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_103 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 111, 111, 32) 96          conv2d_103[0][0]                 
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 111, 111, 32) 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_104

__________________________________________________________________________________________________
mixed0 (Concatenate)            (None, 25, 25, 256)  0           activation_100[0][0]             
                                                                 activation_102[0][0]             
                                                                 activation_105[0][0]             
                                                                 activation_106[0][0]             
__________________________________________________________________________________________________
conv2d_118 (Conv2D)             (None, 25, 25, 64)   16384       mixed0[0][0]                     
__________________________________________________________________________________________________
batch_normalization_110 (BatchN (None, 25, 25, 64)   192         conv2d_118[0][0]                 
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
batch_normalization_114 (BatchN (None, 25, 25, 64)   192         conv2d_122[0][0]                 
__________________________________________________________________________________________________
batch_normalization_116 (BatchN (None, 25, 25, 64)   192         conv2d_124[0][0]                 
__________________________________________________________________________________________________
batch_normalization_119 (BatchN (None, 25, 25, 96)   288         conv2d_127[0][0]                 
__________________________________________________________________________________________________
batch_normalization_120 (BatchN (None, 25, 25, 64)   192         conv2d_128[0][0]                 
__________________________________________________________________________________________________
activation_114 (Activation)     (None, 25, 25, 64)   0           batch_normalization_114[0][0]    
__________

batch_normalization_132 (BatchN (None, 12, 12, 128)  384         conv2d_140[0][0]                 
__________________________________________________________________________________________________
activation_127 (Activation)     (None, 12, 12, 128)  0           batch_normalization_127[0][0]    
__________________________________________________________________________________________________
activation_132 (Activation)     (None, 12, 12, 128)  0           batch_normalization_132[0][0]    
__________________________________________________________________________________________________
average_pooling2d_13 (AveragePo (None, 12, 12, 768)  0           mixed3[0][0]                     
__________________________________________________________________________________________________
conv2d_133 (Conv2D)             (None, 12, 12, 192)  147456      mixed3[0][0]                     
__________________________________________________________________________________________________
conv2d_136

__________________________________________________________________________________________________
batch_normalization_135 (BatchN (None, 12, 12, 192)  576         conv2d_143[0][0]                 
__________________________________________________________________________________________________
batch_normalization_138 (BatchN (None, 12, 12, 192)  576         conv2d_146[0][0]                 
__________________________________________________________________________________________________
batch_normalization_143 (BatchN (None, 12, 12, 192)  576         conv2d_151[0][0]                 
__________________________________________________________________________________________________
batch_normalization_144 (BatchN (None, 12, 12, 192)  576         conv2d_152[0][0]                 
__________________________________________________________________________________________________
activation_135 (Activation)     (None, 12, 12, 192)  0           batch_normalization_135[0][0]    
__________

activation_154 (Activation)     (None, 12, 12, 192)  0           batch_normalization_154[0][0]    
__________________________________________________________________________________________________
mixed6 (Concatenate)            (None, 12, 12, 768)  0           activation_145[0][0]             
                                                                 activation_148[0][0]             
                                                                 activation_153[0][0]             
                                                                 activation_154[0][0]             
__________________________________________________________________________________________________
conv2d_167 (Conv2D)             (None, 12, 12, 192)  147456      mixed6[0][0]                     
__________________________________________________________________________________________________
batch_normalization_159 (BatchN (None, 12, 12, 192)  576         conv2d_167[0][0]                 
__________

batch_normalization_168 (BatchN (None, 12, 12, 192)  576         conv2d_176[0][0]                 
__________________________________________________________________________________________________
activation_168 (Activation)     (None, 12, 12, 192)  0           batch_normalization_168[0][0]    
__________________________________________________________________________________________________
conv2d_173 (Conv2D)             (None, 12, 12, 192)  147456      mixed7[0][0]                     
__________________________________________________________________________________________________
conv2d_177 (Conv2D)             (None, 12, 12, 192)  258048      activation_168[0][0]             
__________________________________________________________________________________________________
batch_normalization_165 (BatchN (None, 12, 12, 192)  576         conv2d_173[0][0]                 
__________________________________________________________________________________________________
batch_norm

activation_178 (Activation)     (None, 5, 5, 384)    0           batch_normalization_178[0][0]    
__________________________________________________________________________________________________
batch_normalization_179 (BatchN (None, 5, 5, 192)    576         conv2d_187[0][0]                 
__________________________________________________________________________________________________
activation_171 (Activation)     (None, 5, 5, 320)    0           batch_normalization_171[0][0]    
__________________________________________________________________________________________________
mixed9_0 (Concatenate)          (None, 5, 5, 768)    0           activation_173[0][0]             
                                                                 activation_174[0][0]             
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 5, 5, 768)    0           activation_177[0][0]             
          

                                                                 mixed9_1[0][0]                   
                                                                 concatenate_4[0][0]              
                                                                 activation_188[0][0]             
__________________________________________________________________________________________________
dropout_17 (Dropout)            (None, 5, 5, 2048)   0           mixed10[0][0]                    
__________________________________________________________________________________________________
dropout_18 (Dropout)            (None, 5, 5, 2048)   0           mixed10[0][0]                    
__________________________________________________________________________________________________
dropout_19 (Dropout)            (None, 5, 5, 2048)   0           mixed10[0][0]                    
__________________________________________________________________________________________________
dropout_20

In [65]:
model2.compile(loss=losses,
              optimizer=Adam(lr=0.01),loss_weights=lossWeights,
              metrics=['accuracy'])

In [66]:
from keras.callbacks import *

clr_triangular = CyclicLR(mode='triangular')
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=2, save_best_only=True, mode='min')
callbacks_list = [checkpoint,clr_triangular]

In [68]:
########1st Iter for 200 epoch



model2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    epochs=200,
    verbose=2,
    callbacks=callbacks_list)

Epoch 1/200
 - 86s - loss: 7.9121 - gender_output_loss: 0.6904 - image_quality_output_loss: 0.9882 - age_output_loss: 1.4503 - weight_output_loss: 0.9936 - bag_output_loss: 0.9306 - footwear_output_loss: 0.9903 - pose_output_loss: 0.9408 - emotion_output_loss: 0.9280 - gender_output_acc: 0.5266 - image_quality_output_acc: 0.5503 - age_output_acc: 0.3911 - weight_output_acc: 0.6345 - bag_output_acc: 0.5523 - footwear_output_acc: 0.5242 - pose_output_acc: 0.6161 - emotion_output_acc: 0.7117 - val_loss: 7.8591 - val_gender_output_loss: 0.6845 - val_image_quality_output_loss: 0.9886 - val_age_output_loss: 1.4303 - val_weight_output_loss: 1.0041 - val_bag_output_loss: 0.9272 - val_footwear_output_loss: 0.9868 - val_pose_output_loss: 0.9211 - val_emotion_output_loss: 0.9165 - val_gender_output_acc: 0.5531 - val_image_quality_output_acc: 0.5432 - val_age_output_acc: 0.3993 - val_weight_output_acc: 0.6379 - val_bag_output_acc: 0.5590 - val_footwear_output_acc: 0.5218 - val_pose_output_acc: 0.6


Epoch 00008: loss improved from 7.43092 to 7.35635, saving model to model.h5
Epoch 9/200
 - 84s - loss: 7.2633 - gender_output_loss: 0.5775 - image_quality_output_loss: 0.9743 - age_output_loss: 1.4004 - weight_output_loss: 0.9754 - bag_output_loss: 0.8636 - footwear_output_loss: 0.8315 - pose_output_loss: 0.7537 - emotion_output_loss: 0.8869 - gender_output_acc: 0.6345 - image_quality_output_acc: 0.5549 - age_output_acc: 0.3964 - weight_output_acc: 0.6337 - bag_output_acc: 0.6020 - footwear_output_acc: 0.6291 - pose_output_acc: 0.6428 - emotion_output_acc: 0.7115 - val_loss: 7.3911 - val_gender_output_loss: 0.5850 - val_image_quality_output_loss: 0.9794 - val_age_output_loss: 1.3957 - val_weight_output_loss: 0.9928 - val_bag_output_loss: 0.9035 - val_footwear_output_loss: 0.8512 - val_pose_output_loss: 0.7794 - val_emotion_output_loss: 0.9042 - val_gender_output_acc: 0.6186 - val_image_quality_output_acc: 0.5437 - val_age_output_acc: 0.4023 - val_weight_output_acc: 0.6374 - val_bag_o


Epoch 00016: loss did not improve from 6.92039
Epoch 17/200
 - 84s - loss: 6.8015 - gender_output_loss: 0.5113 - image_quality_output_loss: 0.9136 - age_output_loss: 1.3813 - weight_output_loss: 0.9637 - bag_output_loss: 0.8163 - footwear_output_loss: 0.7522 - pose_output_loss: 0.6034 - emotion_output_loss: 0.8597 - gender_output_acc: 0.6756 - image_quality_output_acc: 0.5719 - age_output_acc: 0.4002 - weight_output_acc: 0.6352 - bag_output_acc: 0.6430 - footwear_output_acc: 0.6694 - pose_output_acc: 0.7196 - emotion_output_acc: 0.7120 - val_loss: 7.2682 - val_gender_output_loss: 0.5753 - val_image_quality_output_loss: 0.9611 - val_age_output_loss: 1.3819 - val_weight_output_loss: 0.9804 - val_bag_output_loss: 0.8732 - val_footwear_output_loss: 0.8550 - val_pose_output_loss: 0.7323 - val_emotion_output_loss: 0.9090 - val_gender_output_acc: 0.6471 - val_image_quality_output_acc: 0.5184 - val_age_output_acc: 0.4018 - val_weight_output_acc: 0.6379 - val_bag_output_acc: 0.6146 - val_footw


Epoch 00024: loss did not improve from 6.47367
Epoch 25/200
 - 84s - loss: 6.2781 - gender_output_loss: 0.2703 - image_quality_output_loss: 0.8768 - age_output_loss: 1.3554 - weight_output_loss: 0.9449 - bag_output_loss: 0.7865 - footwear_output_loss: 0.7091 - pose_output_loss: 0.4856 - emotion_output_loss: 0.8495 - gender_output_acc: 0.8874 - image_quality_output_acc: 0.5808 - age_output_acc: 0.4068 - weight_output_acc: 0.6359 - bag_output_acc: 0.6638 - footwear_output_acc: 0.6951 - pose_output_acc: 0.8105 - emotion_output_acc: 0.7118 - val_loss: 6.7375 - val_gender_output_loss: 0.3856 - val_image_quality_output_loss: 0.9278 - val_age_output_loss: 1.3907 - val_weight_output_loss: 1.0049 - val_bag_output_loss: 0.8407 - val_footwear_output_loss: 0.7879 - val_pose_output_loss: 0.5201 - val_emotion_output_loss: 0.8798 - val_gender_output_acc: 0.8333 - val_image_quality_output_acc: 0.5556 - val_age_output_acc: 0.4018 - val_weight_output_acc: 0.6384 - val_bag_output_acc: 0.6310 - val_footw


Epoch 00032: loss improved from 5.93228 to 5.73676, saving model to model.h5
Epoch 33/200
 - 84s - loss: 5.8426 - gender_output_loss: 0.1931 - image_quality_output_loss: 0.8633 - age_output_loss: 1.3215 - weight_output_loss: 0.9152 - bag_output_loss: 0.7328 - footwear_output_loss: 0.6651 - pose_output_loss: 0.3116 - emotion_output_loss: 0.8400 - gender_output_acc: 0.9259 - image_quality_output_acc: 0.5905 - age_output_acc: 0.4205 - weight_output_acc: 0.6450 - bag_output_acc: 0.6979 - footwear_output_acc: 0.7147 - pose_output_acc: 0.8888 - emotion_output_acc: 0.7116 - val_loss: 6.5212 - val_gender_output_loss: 0.3607 - val_image_quality_output_loss: 0.8960 - val_age_output_loss: 1.3615 - val_weight_output_loss: 0.9736 - val_bag_output_loss: 0.8362 - val_footwear_output_loss: 0.7737 - val_pose_output_loss: 0.4455 - val_emotion_output_loss: 0.8741 - val_gender_output_acc: 0.8636 - val_image_quality_output_acc: 0.5501 - val_age_output_acc: 0.4053 - val_weight_output_acc: 0.6443 - val_bag_

...


Epoch 00168: loss did not improve from 0.54767
Epoch 169/200
 - 84s - loss: 0.6485 - gender_output_loss: 0.0125 - image_quality_output_loss: 0.0441 - age_output_loss: 0.1565 - weight_output_loss: 0.1407 - bag_output_loss: 0.1362 - footwear_output_loss: 0.0259 - pose_output_loss: 0.0125 - emotion_output_loss: 0.1200 - gender_output_acc: 0.9955 - image_quality_output_acc: 0.9863 - age_output_acc: 0.9276 - weight_output_acc: 0.9231 - bag_output_acc: 0.8990 - footwear_output_acc: 0.9910 - pose_output_acc: 0.9956 - emotion_output_acc: 0.9329 - val_loss: 19.0325 - val_gender_output_loss: 1.0000 - val_image_quality_output_loss: 2.9628 - val_age_output_loss: 3.3067 - val_weight_output_loss: 2.7799 - val_bag_output_loss: 2.4474 - val_footwear_output_loss: 2.5509 - val_pose_output_loss: 1.3243 - val_emotion_output_loss: 2.6607 - val_gender_output_acc: 0.8279 - val_image_quality_output_acc: 0.5387 - val_age_output_acc: 0.3428 - val_weight_output_acc: 0.6171 - val_bag_output_acc: 0.6280 - val_foo


Epoch 00176: loss did not improve from 0.54767
Epoch 177/200
 - 84s - loss: 0.5271 - gender_output_loss: 0.0048 - image_quality_output_loss: 0.0307 - age_output_loss: 0.1303 - weight_output_loss: 0.1193 - bag_output_loss: 0.1217 - footwear_output_loss: 0.0119 - pose_output_loss: 0.0063 - emotion_output_loss: 0.1021 - gender_output_acc: 0.9985 - image_quality_output_acc: 0.9903 - age_output_acc: 0.9354 - weight_output_acc: 0.9274 - bag_output_acc: 0.9016 - footwear_output_acc: 0.9966 - pose_output_acc: 0.9983 - emotion_output_acc: 0.9366 - val_loss: 20.1202 - val_gender_output_loss: 1.0524 - val_image_quality_output_loss: 3.2516 - val_age_output_loss: 3.4825 - val_weight_output_loss: 2.5314 - val_bag_output_loss: 2.8305 - val_footwear_output_loss: 2.7217 - val_pose_output_loss: 1.4955 - val_emotion_output_loss: 2.7545 - val_gender_output_acc: 0.8442 - val_image_quality_output_acc: 0.5432 - val_age_output_acc: 0.3725 - val_weight_output_acc: 0.5367 - val_bag_output_acc: 0.6106 - val_foo


Epoch 00184: loss did not improve from 0.52654
Epoch 185/200
 - 89s - loss: 0.5998 - gender_output_loss: 0.0085 - image_quality_output_loss: 0.0418 - age_output_loss: 0.1460 - weight_output_loss: 0.1302 - bag_output_loss: 0.1294 - footwear_output_loss: 0.0159 - pose_output_loss: 0.0092 - emotion_output_loss: 0.1188 - gender_output_acc: 0.9974 - image_quality_output_acc: 0.9857 - age_output_acc: 0.9319 - weight_output_acc: 0.9248 - bag_output_acc: 0.8998 - footwear_output_acc: 0.9944 - pose_output_acc: 0.9969 - emotion_output_acc: 0.9342 - val_loss: 18.6260 - val_gender_output_loss: 1.0634 - val_image_quality_output_loss: 2.9016 - val_age_output_loss: 3.0937 - val_weight_output_loss: 2.5267 - val_bag_output_loss: 2.5496 - val_footwear_output_loss: 2.6721 - val_pose_output_loss: 1.3197 - val_emotion_output_loss: 2.4993 - val_gender_output_acc: 0.8368 - val_image_quality_output_acc: 0.5298 - val_age_output_acc: 0.3641 - val_weight_output_acc: 0.5813 - val_bag_output_acc: 0.6121 - val_foo


Epoch 00192: loss did not improve from 0.51098
Epoch 193/200
 - 91s - loss: 0.7075 - gender_output_loss: 0.0128 - image_quality_output_loss: 0.0574 - age_output_loss: 0.1718 - weight_output_loss: 0.1433 - bag_output_loss: 0.1388 - footwear_output_loss: 0.0413 - pose_output_loss: 0.0165 - emotion_output_loss: 0.1255 - gender_output_acc: 0.9955 - image_quality_output_acc: 0.9797 - age_output_acc: 0.9235 - weight_output_acc: 0.9201 - bag_output_acc: 0.8986 - footwear_output_acc: 0.9851 - pose_output_acc: 0.9945 - emotion_output_acc: 0.9315 - val_loss: 18.2722 - val_gender_output_loss: 0.9686 - val_image_quality_output_loss: 2.7258 - val_age_output_loss: 3.3142 - val_weight_output_loss: 2.6120 - val_bag_output_loss: 2.4173 - val_footwear_output_loss: 2.4654 - val_pose_output_loss: 1.1448 - val_emotion_output_loss: 2.6240 - val_gender_output_acc: 0.8452 - val_image_quality_output_acc: 0.5407 - val_age_output_acc: 0.3641 - val_weight_output_acc: 0.5962 - val_bag_output_acc: 0.6314 - val_foo


Epoch 00200: loss did not improve from 0.51098
